In [1]:
import sys
from pathlib import Path

home = str(Path.home())
core_dir = home+'/repositories/ai-x/core'
sys.path.insert(0, core_dir)
selectivity_dir = home+'/repositories/ai-x/core/selectivity'
sys.path.insert(0, selectivity_dir)
#from selectivity import *
from misc import check_output_dir
import pandas as pd
import numpy as np

output_dir = "output_dir"
check_output_dir(output_dir, keep_old = False)     

datadir = home+"/repositories/ai-DR/selectivity_models/models_C33_200/"
datadir

'/Users/sjwon3789/repositories/ai-DR/selectivity_models/models_C33_200/'

# Internal

In [2]:
from sklearn.metrics import recall_score, accuracy_score, f1_score, roc_auc_score

def stats(test_acts, test_results):
    '''Test_acts is the true value, test_results is predicted value'''
    acc = accuracy_score(test_acts, test_results)
    sens = recall_score(test_acts, test_results, pos_label=1)
    spec = recall_score(test_acts, test_results, pos_label=0)
    f1 = f1_score(test_acts, test_results)
    auc = roc_auc_score(test_acts, test_results)

    return acc, sens, spec, f1, auc


def predict_method(df, method):
    y_pred = list(df[method])
    y_true = list(df['exp_mean'])

    acc, sens, spec, f1, auc = stats(y_true, y_pred)
    return acc, sens, spec, f1, auc

# Define a list of model directories
model_directories = ["models_D2", "models_D3", "models__ratio_D2D3"]

In [ ]:
%%time

ls_methods = ["xgb", "rf", "dnn"]  # Add DNN to the methods

# Create a dictionary to store results
results = []

model_dir = "models_D3"
# Loop through each model directory

print(model_dir)
acc_values_xgb = []
sens_values_xgb = []
spec_values_xgb = []
f1_values_xgb = []
auc_values_xgb = []

acc_values_rf = []
sens_values_rf = []
spec_values_rf = []
f1_values_rf = []
auc_values_rf = []

acc_values_dnn = []  # Add lists for DNN
sens_values_dnn = []
spec_values_dnn = []
f1_values_dnn = []
auc_values_dnn = []

for i in range(10):
    print(i)
    df1 = pd.read_csv(datadir + f"{model_dir}/model_{i}/class_xgb_0.15/pred_864.csv", sep="\t", low_memory=True)
    df1 = df1[['compound', 'exp_mean', 'pred_864']]
    df2 = pd.read_csv(datadir + f"{model_dir}/model_{i}/class_rf_0.15/pred_864.csv", sep="\t")
    df2 = df2[['compound', "exp_mean", 'pred_864']]
    df3 = pd.read_csv(datadir + f"{model_dir}/model_{i}/class_dnn_0.15/pred_864.csv", sep="\t")  # Add DNN predictions
    df3 = df3[['compound', 'exp_mean', 'pred_864']]

    # Rename the columns to match method names
    df1.rename(columns={'pred_864': 'xgb'}, inplace=True)
    df2.rename(columns={'pred_864': 'rf'}, inplace=True)
    df3.rename(columns={'pred_864': 'dnn'}, inplace=True)

    # Merge the data frames based on the common columns 'compound' and 'exp_mean', then take majority voting
    merged_df = pd.merge(df1, df2, on=['compound', 'exp_mean'], how='outer')
    merged_df = pd.merge(merged_df, df3, on=['compound', 'exp_mean'], how='outer')  # Merge DNN predictions

    # Calculate statistical measures per each method and store them
    acc, sens, spec, f1, auc = predict_method(merged_df, "xgb")
    acc_values_xgb.append(acc)
    sens_values_xgb.append(sens)
    spec_values_xgb.append(spec)
    f1_values_xgb.append(f1)
    auc_values_xgb.append(auc)
    
    acc, sens, spec, f1, auc = predict_method(merged_df, "rf")
    acc_values_rf.append(acc)
    sens_values_rf.append(sens)
    spec_values_rf.append(spec)
    f1_values_rf.append(f1)
    auc_values_rf.append(auc)
    
    acc, sens, spec, f1, auc = predict_method(merged_df, "dnn")  # Add DNN predictions
    acc_values_dnn.append(acc)
    sens_values_dnn.append(sens)
    spec_values_dnn.append(spec)
    f1_values_dnn.append(f1)
    auc_values_dnn.append(auc)
    
    
# Create a data frame to store the results for the current model
model_results_df = pd.DataFrame({
    'acc_mean': [
        f"{np.mean(acc_values_xgb):.2f} ({np.std(acc_values_xgb):.2f})",
        f"{np.mean(acc_values_rf):.2f} ({np.std(acc_values_rf):.2f})",
        f"{np.mean(acc_values_dnn):.2f} ({np.std(acc_values_dnn):.2f})"
    ],
    'sens_mean': [
        f"{np.mean(sens_values_xgb):.2f} ({np.std(sens_values_xgb):.2f})",
        f"{np.mean(sens_values_rf):.2f} ({np.std(sens_values_rf):.2f})",
        f"{np.mean(sens_values_dnn):.2f} ({np.std(sens_values_dnn):.2f})"
    ],
    'spec_mean': [
        f"{np.mean(spec_values_xgb):.2f} ({np.std(spec_values_xgb):.2f})",
        f"{np.mean(spec_values_rf):.2f} ({np.std(spec_values_rf):.2f})",
        f"{np.mean(spec_values_dnn):.2f} ({np.std(spec_values_dnn):.2f})"
    ],
    'f1_mean': [
        f"{np.mean(f1_values_xgb):.2f} ({np.std(f1_values_xgb):.2f})",
        f"{np.mean(f1_values_rf):.2f} ({np.std(f1_values_rf):.2f})",
        f"{np.mean(f1_values_dnn):.2f} ({np.std(f1_values_dnn):.2f})"
    ],
    'auc_mean': [
        f"{np.mean(auc_values_xgb):.2f} ({np.std(auc_values_xgb):.2f})",
        f"{np.mean(auc_values_rf):.2f} ({np.std(auc_values_rf):.2f})",
        f"{np.mean(auc_values_dnn):.2f} ({np.std(auc_values_dnn):.2f})"
    ],
}, index=['xgb', 'rf', 'dnn'])  # Add DNN to the index

# Append the results for the current model to the overall results list
results.append((model_dir, model_results_df))

# Create a data frame to store the results for all models
final_results_df = pd.concat([result[1] for result in results], keys=[result[0] for result in results])
# final_results_df.to_csv(f"output_dir/consensus_classification_internal_{model}.csv")
final_results_df


In [4]:
%%time

# List of model directories
model_directories = ["models_D2", "models_D3", "models__ratio_D2D3"]

# Create a dictionary to store results
results = []

# Initialize lists for performance metrics for each model directory
for model_dir in model_directories:
    print(f"Processing directory: {model_dir}")

    # Initialize empty lists for each model's performance metrics
    acc_values_xgb = []
    sens_values_xgb = []
    spec_values_xgb = []
    f1_values_xgb = []
    auc_values_xgb = []

    acc_values_rf = []
    sens_values_rf = []
    spec_values_rf = []
    f1_values_rf = []
    auc_values_rf = []

    acc_values_dnn = []  # Add lists for DNN
    sens_values_dnn = []
    spec_values_dnn = []
    f1_values_dnn = []
    auc_values_dnn = []

    for i in range(10):
        print(f"  Processing model {i}")
        
        # Load the prediction data for each model (XGB, RF, and DNN)
        df1 = pd.read_csv(datadir + f"{model_dir}/model_{i}/class_xgb_0.15/pred_864.csv", sep="\t", low_memory=True)
        df1 = df1[['compound', 'exp_mean', 'pred_864']]
        
        df2 = pd.read_csv(datadir + f"{model_dir}/model_{i}/class_rf_0.15/pred_864.csv", sep="\t")
        df2 = df2[['compound', "exp_mean", 'pred_864']]
        
        df3 = pd.read_csv(datadir + f"{model_dir}/model_{i}/class_dnn_0.15/pred_864.csv", sep="\t")  # Add DNN predictions
        df3 = df3[['compound', 'exp_mean', 'pred_864']]

        # Rename the columns to match method names
        df1.rename(columns={'pred_864': 'xgb'}, inplace=True)
        df2.rename(columns={'pred_864': 'rf'}, inplace=True)
        df3.rename(columns={'pred_864': 'dnn'}, inplace=True)
        
        # Calculate performance metrics separately for each model

        # XGB
        acc, sens, spec, f1, auc = predict_method(df1, "xgb")
        acc_values_xgb.append(acc)
        sens_values_xgb.append(sens)
        spec_values_xgb.append(spec)
        f1_values_xgb.append(f1)
        auc_values_xgb.append(auc)
        
        # RF
        acc, sens, spec, f1, auc = predict_method(df2, "rf")
        acc_values_rf.append(acc)
        sens_values_rf.append(sens)
        spec_values_rf.append(spec)
        f1_values_rf.append(f1)
        auc_values_rf.append(auc)
        
        # DNN
        acc, sens, spec, f1, auc = predict_method(df3, "dnn")  # Add DNN predictions
        acc_values_dnn.append(acc)
        sens_values_dnn.append(sens)
        spec_values_dnn.append(spec)
        f1_values_dnn.append(f1)
        auc_values_dnn.append(auc)

    # Create a data frame to store the results for the current model directory
    model_results_df = pd.DataFrame({
        'acc_mean': [
            f"{np.mean(acc_values_xgb):.2f} ({np.std(acc_values_xgb):.2f})",
            f"{np.mean(acc_values_rf):.2f} ({np.std(acc_values_rf):.2f})",
            f"{np.mean(acc_values_dnn):.2f} ({np.std(acc_values_dnn):.2f})"
        ],
        'sens_mean': [
            f"{np.mean(sens_values_xgb):.2f} ({np.std(sens_values_xgb):.2f})",
            f"{np.mean(sens_values_rf):.2f} ({np.std(sens_values_rf):.2f})",
            f"{np.mean(sens_values_dnn):.2f} ({np.std(sens_values_dnn):.2f})"
        ],
        'spec_mean': [
            f"{np.mean(spec_values_xgb):.2f} ({np.std(spec_values_xgb):.2f})",
            f"{np.mean(spec_values_rf):.2f} ({np.std(spec_values_rf):.2f})",
            f"{np.mean(spec_values_dnn):.2f} ({np.std(spec_values_dnn):.2f})"
        ],
        'f1_mean': [
            f"{np.mean(f1_values_xgb):.2f} ({np.std(f1_values_xgb):.2f})",
            f"{np.mean(f1_values_rf):.2f} ({np.std(f1_values_rf):.2f})",
            f"{np.mean(f1_values_dnn):.2f} ({np.std(f1_values_dnn):.2f})"
        ],
        'auc_mean': [
            f"{np.mean(auc_values_xgb):.2f} ({np.std(auc_values_xgb):.2f})",
            f"{np.mean(auc_values_rf):.2f} ({np.std(auc_values_rf):.2f})",
            f"{np.mean(auc_values_dnn):.2f} ({np.std(auc_values_dnn):.2f})"
        ],
    }, index=['xgb', 'rf', 'dnn'])  # Add DNN to the index

    # Append the results for the current model directory to the overall results list
    results.append((model_dir, model_results_df))

# Create a data frame to store the results for all model directories
final_results_df = pd.concat([result[1] for result in results], keys=[result[0] for result in results])

# final_results_df.to_csv(f"output_dir/consensus_classification_internal_all.csv")
final_results_df


Processing directory: models_D2
  Processing model 0
  Processing model 1
  Processing model 2
  Processing model 3
  Processing model 4
  Processing model 5
  Processing model 6
  Processing model 7
  Processing model 8
  Processing model 9
Processing directory: models_D3
  Processing model 0
  Processing model 1
  Processing model 2
  Processing model 3
  Processing model 4
  Processing model 5
  Processing model 6
  Processing model 7
  Processing model 8
  Processing model 9
Processing directory: models__ratio_D2D3
  Processing model 0
  Processing model 1
  Processing model 2
  Processing model 3
  Processing model 4
  Processing model 5
  Processing model 6
  Processing model 7
  Processing model 8
  Processing model 9
CPU times: user 258 ms, sys: 9.69 ms, total: 268 ms
Wall time: 281 ms


acc_mean    sens_mean    spec_mean      f1_mean  \
models_D2          xgb  0.94 (0.01)  0.95 (0.01)  0.79 (0.10)  0.97 (0.01)   
                   rf   0.95 (0.01)  0.96 (0.01)  0.73 (0.11)  0.97 (0.01)   
                   dnn  0.91 (0.08)  0.93 (0.09)  0.66 (0.26)  0.95 (0.05)   
models_D3          xgb  0.95 (0.01)  0.96 (0.01)  0.85 (0.17)  0.98 (0.01)   
                   rf   0.95 (0.02)  0.95 (0.01)  0.86 (0.20)  0.97 (0.01)   
                   dnn  0.94 (0.02)  0.95 (0.02)  0.69 (0.31)  0.97 (0.01)   
models__ratio_D2D3 xgb  0.93 (0.02)  0.93 (0.01)  0.93 (0.03)  0.93 (0.02)   
                   rf   0.93 (0.02)  0.92 (0.02)  0.93 (0.03)  0.93 (0.02)   
                   dnn  0.92 (0.02)  0.92 (0.02)  0.92 (0.03)  0.93 (0.02)   

                           auc_mean  
models_D2          xgb  0.87 (0.05)  
                   rf   0.84 (0.06)  
                   dnn  0.80 (0.11)  
models_D3          xgb  0.90 (0.08)  
                   rf   0.90 (0.10)  
                   dnn  0.82 (0.15)  
models__ratio_D2D3 xgb  0.93 (0.02)  
                   rf   0.93 (0.02)  
                   dnn  0.92 (0.02)

In [5]:
df1

,compound,exp_mean,xgb
0,CHEMBL54,0,0
1,CHEMBL71,0,0
2,CHEMBL85,0,0
3,CHEMBL589,1,1
4,CHEMBL629,1,0
...,...,...,...
152,CHEMBL231431,0,0
153,CHEMBL231546,0,0
154,CHEMBL231640,0,0
155,CHEMBL231648,1,1


In [6]:
df3

,compound,exp_mean,dnn
0,CHEMBL596,0,0
1,CHEMBL6437,0,0
2,CHEMBL8946,0,0
3,CHEMBL25236,1,1
4,CHEMBL25856,1,0
...,...,...,...
152,CHEMBL4854935,0,0
153,CHEMBL4857166,0,0
154,CHEMBL4876207,0,0
155,CHEMBL4877715,1,1


In [7]:
merged_df

,compound,exp_mean,xgb,rf,dnn
0,CHEMBL143761,0,0.0,0.0,NaN
1,CHEMBL4435834,0,0.0,0.0,NaN
2,CHEMBL272602,0,1.0,1.0,NaN
3,CHEMBL4536530,0,0.0,0.0,NaN
4,CHEMBL3622103,0,1.0,1.0,NaN
...,...,...,...,...,...
505,CHEMBL4868595,1,NaN,NaN,1.0
506,CHEMBL4875181,1,NaN,NaN,1.0
507,CHEMBL4878501,1,NaN,NaN,1.0
508,CHEMBL4878826,1,NaN,NaN,1.0


# External Validation

In [8]:
# Define the list of model directories
model_directories = ["models_D2", "models_D3", "models__ratio_D2D3"]

# Define a list of methods
ls_methods = ["xgb", "rf", "dnn"]

# Initialize a list to store results for all models
results = []

# Loop through each model directory
for model_dir in model_directories:
    # Create empty lists to store performance metrics for each method
    acc_values_xgb = []
    sens_values_xgb = []
    spec_values_xgb = []
    f1_values_xgb = []
    auc_values_xgb = []

    acc_values_rf = []
    sens_values_rf = []
    spec_values_rf = []
    f1_values_rf = []
    auc_values_rf = []

    acc_values_dnn = []
    sens_values_dnn = []
    spec_values_dnn = []
    f1_values_dnn = []
    auc_values_dnn = []

    for i in range(0, 10):
        print(i)
        # Load the prediction data for each model (XGB, RF, and DNN)
        df1 = pd.read_csv(datadir + f"{model_dir}/model_{i}/pred_class_xgb_0.00/pred_splits_val_class{i}", sep="\t")
        df1 = df1[['compound', 'exp_mean', 'pred_activity']]
        
        df2 = pd.read_csv(datadir + f"{model_dir}/model_{i}/pred_class_rf_0.00/pred_splits_val_class{i}", sep="\t")
        df2 = df2[['compound', 'exp_mean', 'pred_activity']]
        
        df3 = pd.read_csv(datadir + f"{model_dir}/model_{i}/pred_class_dnn_0.00/pred_splits", sep="\t")
        df3 = df3[['compound', 'exp_mean', 'pred_activity']]

        # Rename the 'pred_activity' columns to model names for clarity
        df1.rename(columns={'pred_activity': 'xgb'}, inplace=True)
        df2.rename(columns={'pred_activity': 'rf'}, inplace=True)
        df3.rename(columns={'pred_activity': 'dnn'}, inplace=True)
        
        # Merge the data frames based on the common columns 'compound' and 'exp_mean'
        merged_df = pd.merge(df1, df2, on=['compound', 'exp_mean'], how='outer')
        merged_df = pd.merge(merged_df, df3, on=['compound', 'exp_mean'], how='outer')

        # Calculate performance metrics for each model and append to corresponding lists
        acc, sens, spec, f1, auc = predict_method(merged_df, "xgb")
        acc_values_xgb.append(acc)
        sens_values_xgb.append(sens)
        spec_values_xgb.append(spec)
        f1_values_xgb.append(f1)
        auc_values_xgb.append(auc)

        acc, sens, spec, f1, auc = predict_method(merged_df, "rf")
        acc_values_rf.append(acc)
        sens_values_rf.append(sens)
        spec_values_rf.append(spec)
        f1_values_rf.append(f1)
        auc_values_rf.append(auc)

        acc, sens, spec, f1, auc = predict_method(merged_df, "dnn")
        acc_values_dnn.append(acc)
        sens_values_dnn.append(sens)
        spec_values_dnn.append(spec)
        f1_values_dnn.append(f1)
        auc_values_dnn.append(auc)

    # Create a data frame to store the results for the current model
    model_results_df = pd.DataFrame({
        'acc_mean': [f"{np.mean(acc_values_xgb):.2f} ({np.std(acc_values_xgb):.2f})",
                    f"{np.mean(acc_values_rf):.2f} ({np.std(acc_values_rf):.2f})",
                    f"{np.mean(acc_values_dnn):.2f} ({np.std(acc_values_dnn):.2f})"],
        'sens_mean': [f"{np.mean(sens_values_xgb):.2f} ({np.std(sens_values_xgb):.2f})",
                    f"{np.mean(sens_values_rf):.2f} ({np.std(sens_values_rf):.2f})",
                    f"{np.mean(sens_values_dnn):.2f} ({np.std(sens_values_dnn):.2f})"],
        'spec_mean': [f"{np.mean(spec_values_xgb):.2f} ({np.std(spec_values_xgb):.2f})",
                    f"{np.mean(spec_values_rf):.2f} ({np.std(spec_values_rf):.2f})",
                    f"{np.mean(spec_values_dnn):.2f} ({np.std(spec_values_dnn):.2f})"],
        'f1_mean': [f"{np.mean(f1_values_xgb):.2f} ({np.std(f1_values_xgb):.2f})",
                    f"{np.mean(f1_values_rf):.2f} ({np.std(f1_values_rf):.2f})",
                    f"{np.mean(f1_values_dnn):.2f} ({np.std(f1_values_dnn):.2f})"],
        'auc_mean': [f"{np.mean(auc_values_xgb):.2f} ({np.std(auc_values_xgb):.2f})",
                    f"{np.mean(auc_values_rf):.2f} ({np.std(auc_values_rf):.2f})",
                    f"{np.mean(auc_values_dnn):.2f} ({np.std(auc_values_dnn):.2f})"],
    }, index=['xgb', 'rf', 'dnn'])

    # Append the results for the current model to the overall results list
    results.append((model_dir, model_results_df))

# Create a data frame to store the results for all models
final_results_df = pd.concat([result[1] for result in results], keys=[result[0] for result in results])

# Save the results to a CSV file
# final_results_df.to_csv("output_dir/consensus_classification_external_with_dnn.csv")

# Display the final results
final_results_df


0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


acc_mean    sens_mean    spec_mean      f1_mean  \
models_D2          xgb  0.91 (0.02)  0.95 (0.02)  0.76 (0.07)  0.95 (0.02)   
                   rf   0.91 (0.01)  0.96 (0.02)  0.75 (0.06)  0.95 (0.01)   
                   dnn  0.75 (0.25)  0.77 (0.37)  0.63 (0.32)  0.75 (0.35)   
models_D3          xgb  0.93 (0.02)  0.96 (0.01)  0.83 (0.07)  0.96 (0.01)   
                   rf   0.93 (0.02)  0.96 (0.01)  0.81 (0.06)  0.96 (0.01)   
                   dnn  0.76 (0.20)  0.74 (0.28)  0.84 (0.12)  0.80 (0.21)   
models__ratio_D2D3 xgb  0.93 (0.02)  0.92 (0.02)  0.94 (0.03)  0.94 (0.01)   
                   rf   0.94 (0.02)  0.93 (0.03)  0.95 (0.04)  0.94 (0.02)   
                   dnn  0.93 (0.02)  0.94 (0.03)  0.93 (0.03)  0.94 (0.02)   

                           auc_mean  
models_D2          xgb  0.86 (0.03)  
                   rf   0.85 (0.03)  
                   dnn  0.70 (0.16)  
models_D3          xgb  0.89 (0.04)  
                   rf   0.89 (0.03)  
                   dnn  0.79 (0.11)  
models__ratio_D2D3 xgb  0.93 (0.02)  
                   rf   0.94 (0.02)  
                   dnn  0.93 (0.02)

# Consensus for classification 100%

In [9]:
final_results_df# Define the list of model directories
model_directories = ["models_D2", "models_D3", "models__ratio_D2D3"]

# Define a list of methods
ls_methods = ["xgb", "rf", "dnn"]

# Initialize a list to store results for all models
results = []

# Loop through each model directory
for model_dir in model_directories:
    # Create empty lists to store performance metrics for each method
    acc_values_xgb = []
    sens_values_xgb = []
    spec_values_xgb = []
    f1_values_xgb = []
    auc_values_xgb = []

    acc_values_rf = []
    sens_values_rf = []
    spec_values_rf = []
    f1_values_rf = []
    auc_values_rf = []

    acc_values_dnn = []
    sens_values_dnn = []
    spec_values_dnn = []
    f1_values_dnn = []
    auc_values_dnn = []

    acc_values_consensus = []
    sens_values_consensus = []
    spec_values_consensus = []
    f1_values_consensus = []
    auc_values_consensus = []

    for i in range(0, 10):
        print(i)
        # Load the prediction data for each model (XGB, RF, and DNN)
        df1 = pd.read_csv(datadir + f"{model_dir}/model_{i}/pred_class_xgb_0.00/pred_splits_val_class{i}", sep="\t")
        df1 = df1[['compound', 'exp_mean', 'pred_activity']]
        
        df2 = pd.read_csv(datadir + f"{model_dir}/model_{i}/pred_class_rf_0.00/pred_splits_val_class{i}", sep="\t")
        df2 = df2[['compound', 'exp_mean', 'pred_activity']]
        
        df3 = pd.read_csv(datadir + f"{model_dir}/model_{i}/pred_class_dnn_0.00/pred_splits", sep="\t")
        df3 = df3[['compound', 'exp_mean', 'pred_activity']]

        # Rename the 'pred_activity' columns to model names for clarity
        df1.rename(columns={'pred_activity': 'xgb'}, inplace=True)
        df2.rename(columns={'pred_activity': 'rf'}, inplace=True)
        df3.rename(columns={'pred_activity': 'dnn'}, inplace=True)
        
        # Merge the data frames based on the common columns 'compound' and 'exp_mean'
        merged_df = pd.merge(df1, df2, on=['compound', 'exp_mean'], how='outer')
        merged_df = pd.merge(merged_df, df3, on=['compound', 'exp_mean'], how='outer')

        # Calculate consensus prediction (majority voting: 2 out of 3)
        merged_df['consensus'] = merged_df[['xgb', 'rf', 'dnn']].mode(axis=1)[0]

        # Calculate performance metrics for each model and consensus
        acc, sens, spec, f1, auc = predict_method(merged_df, "xgb")
        acc_values_xgb.append(acc)
        sens_values_xgb.append(sens)
        spec_values_xgb.append(spec)
        f1_values_xgb.append(f1)
        auc_values_xgb.append(auc)

        acc, sens, spec, f1, auc = predict_method(merged_df, "rf")
        acc_values_rf.append(acc)
        sens_values_rf.append(sens)
        spec_values_rf.append(spec)
        f1_values_rf.append(f1)
        auc_values_rf.append(auc)

        acc, sens, spec, f1, auc = predict_method(merged_df, "dnn")
        acc_values_dnn.append(acc)
        sens_values_dnn.append(sens)
        spec_values_dnn.append(spec)
        f1_values_dnn.append(f1)
        auc_values_dnn.append(auc)

        # Calculate performance metrics for consensus
        acc, sens, spec, f1, auc = predict_method(merged_df, "consensus")
        acc_values_consensus.append(acc)
        sens_values_consensus.append(sens)
        spec_values_consensus.append(spec)
        f1_values_consensus.append(f1)
        auc_values_consensus.append(auc)

    # Create a data frame to store the results for the current model
    model_results_df = pd.DataFrame({
        'acc_mean': [f"{np.mean(acc_values_xgb):.2f} ({np.std(acc_values_xgb):.2f})",
                    f"{np.mean(acc_values_rf):.2f} ({np.std(acc_values_rf):.2f})",
                    f"{np.mean(acc_values_dnn):.2f} ({np.std(acc_values_dnn):.2f})",
                    f"{np.mean(acc_values_consensus):.2f} ({np.std(acc_values_consensus):.2f})"],
        'sens_mean': [f"{np.mean(sens_values_xgb):.2f} ({np.std(sens_values_xgb):.2f})",
                    f"{np.mean(sens_values_rf):.2f} ({np.std(sens_values_rf):.2f})",
                    f"{np.mean(sens_values_dnn):.2f} ({np.std(sens_values_dnn):.2f})",
                    f"{np.mean(sens_values_consensus):.2f} ({np.std(sens_values_consensus):.2f})"],
        'spec_mean': [f"{np.mean(spec_values_xgb):.2f} ({np.std(spec_values_xgb):.2f})",
                    f"{np.mean(spec_values_rf):.2f} ({np.std(spec_values_rf):.2f})",
                    f"{np.mean(spec_values_dnn):.2f} ({np.std(spec_values_dnn):.2f})",
                    f"{np.mean(spec_values_consensus):.2f} ({np.std(spec_values_consensus):.2f})"],
        'f1_mean': [f"{np.mean(f1_values_xgb):.2f} ({np.std(f1_values_xgb):.2f})",
                    f"{np.mean(f1_values_rf):.2f} ({np.std(f1_values_rf):.2f})",
                    f"{np.mean(f1_values_dnn):.2f} ({np.std(f1_values_dnn):.2f})",
                    f"{np.mean(f1_values_consensus):.2f} ({np.std(f1_values_consensus):.2f})"],
        'auc_mean': [f"{np.mean(auc_values_xgb):.2f} ({np.std(auc_values_xgb):.2f})",
                    f"{np.mean(auc_values_rf):.2f} ({np.std(auc_values_rf):.2f})",
                    f"{np.mean(auc_values_dnn):.2f} ({np.std(auc_values_dnn):.2f})",
                    f"{np.mean(auc_values_consensus):.2f} ({np.std(auc_values_consensus):.2f})"],
    }, index=['xgb', 'rf', 'dnn', 'consensus'])

    # Append the results for the current model to the overall results list
    results.append((model_dir, model_results_df))

# Create a data frame to store the results for all models
final_results_df = pd.concat([result[1] for result in results], keys=[result[0] for result in results])

# Save the results to a CSV file
final_results_df.to_csv("output_dir/consensus_classification_external_with_dnn_and_consensus.csv")

# Display the final results
final_results_df


0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


acc_mean    sens_mean    spec_mean  \
models_D2          xgb        0.91 (0.02)  0.95 (0.02)  0.76 (0.07)   
                   rf         0.91 (0.01)  0.96 (0.02)  0.75 (0.06)   
                   dnn        0.75 (0.25)  0.77 (0.37)  0.63 (0.32)   
                   consensus  0.91 (0.02)  0.96 (0.02)  0.75 (0.07)   
models_D3          xgb        0.93 (0.02)  0.96 (0.01)  0.83 (0.07)   
                   rf         0.93 (0.02)  0.96 (0.01)  0.81 (0.06)   
                   dnn        0.76 (0.20)  0.74 (0.28)  0.84 (0.12)   
                   consensus  0.94 (0.02)  0.96 (0.01)  0.83 (0.07)   
models__ratio_D2D3 xgb        0.93 (0.02)  0.92 (0.02)  0.94 (0.03)   
                   rf         0.94 (0.02)  0.93 (0.03)  0.95 (0.04)   
                   dnn        0.93 (0.02)  0.94 (0.03)  0.93 (0.03)   
                   consensus  0.93 (0.02)  0.93 (0.03)  0.94 (0.03)   

                                  f1_mean     auc_mean  
models_D2          xgb        0.95 (0.02)  0.86 (0.03)  
                   rf         0.95 (0.01)  0.85 (0.03)  
                   dnn        0.75 (0.35)  0.70 (0.16)  
                   consensus  0.95 (0.01)  0.85 (0.03)  
models_D3          xgb        0.96 (0.01)  0.89 (0.04)  
                   rf         0.96 (0.01)  0.89 (0.03)  
                   dnn        0.80 (0.21)  0.79 (0.11)  
                   consensus  0.96 (0.01)  0.90 (0.03)  
models__ratio_D2D3 xgb        0.94 (0.01)  0.93 (0.02)  
                   rf         0.94 (0.02)  0.94 (0.02)  
                   dnn        0.94 (0.02)  0.93 (0.02)  
                   consensus  0.94 (0.01)  0.94 (0.02)